# Preparing Time Series Data

## 1.0 Import Library

In [141]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from scipy import signal

<br />

---

## 2.0 Preview the Sales Data

### 2.1 Load and Preview First Fives Rows

In [142]:
df = pd.read_csv('sales.csv', parse_dates=['date'])

In [143]:
df.head()

,date,store_id,cat_id,sales
0,2011-01-29,TX_1,FOODS,3950.35
1,2011-01-30,TX_1,FOODS,3844.97
2,2011-01-31,TX_1,FOODS,2888.03
3,2011-02-01,TX_1,FOODS,3631.28
4,2011-02-02,TX_1,FOODS,3072.18


In [144]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58230 entries, 0 to 58229
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   date      58230 non-null  datetime64[ns]
 1   store_id  58230 non-null  object        
 2   cat_id    58230 non-null  object        
 3   sales     58230 non-null  float64       
dtypes: datetime64[ns](1), float64(1), object(2)
memory usage: 1.8+ MB


<br />

### 2.2 Number of Unique Values of `store_id` and `cat_id`

In [145]:
df[['store_id', 'cat_id']].nunique()

store_id    10
cat_id       3
dtype: int64

<br />

### 2.3 Date Range of the `sales` data

In [146]:
start_date = df['date'].min()
end_date = df['date'].max()

In [147]:
('Start Date', start_date), ('End Date', end_date)

(('Start Date', Timestamp('2011-01-29 00:00:00')),
 ('End Date', Timestamp('2016-05-22 00:00:00')))

The date range of the `sales` data is from 29 Jan 2011 to 22 May 2016

<br />

---

## 3.0 Process the Data

### 3.1 Duplicated Dates Within A Store and Category Group

In [148]:
df[df.duplicated(['date', 'store_id', 'cat_id'])]

,date,store_id,cat_id,sales


Nope, there isn't any duplicated dates within a store and category group

<br />

### 3.2 Check for Missing dates

In [149]:
daily_df = pd.date_range(start=start_date, end=end_date).to_frame(name='daily_date')

In [150]:
pd.merge(daily_df, df, left_index=True, right_on='date', how='left').isnull().sum()

daily_date    0
date          0
store_id      0
cat_id        0
sales         0
dtype: int64

There isn't any missing value in the `sales` data

<br />

### 3.3 Check Outliners with IQR method

#### 3.3.1 Detrend `sales` feature

In [151]:
df['sales_detrend'] = signal.detrend(df['sales'])

Reference:
- [Why detrend?](https://www.investopedia.com/terms/d/detrend.asp)

#### 3.3.2 Detect outlier

In [152]:
def detect_outlier(df, col):
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    
    df['is_anomaly'] = (df[col] <= Q1 - 1.5 * IQR) | (df[col] >= Q3 + 1.5 * IQR)
    
    return df

Reference:
- [Why outside 1.5 of IQR is considered outlier?](https://towardsdatascience.com/why-1-5-in-iqr-method-of-outlier-detection-5d07fdc82097)

In [153]:
df = detect_outlier(df, 'sales_detrend')

In [154]:
df.head()

,date,store_id,cat_id,sales,sales_detrend,is_anomaly
0,2011-01-29,TX_1,FOODS,3950.35,299.022047,False
1,2011-01-30,TX_1,FOODS,3844.97,193.654458,False
2,2011-01-31,TX_1,FOODS,2888.03,-763.273132,False
3,2011-02-01,TX_1,FOODS,3631.28,-20.010722,False
4,2011-02-02,TX_1,FOODS,3072.18,-579.098312,False


In [155]:
len(df.loc[df['is_anomaly']])

1623

There are a total of 1623 anomaly instances in the dataset

In [156]:
df.loc[df['is_anomaly']].head()

,date,store_id,cat_id,sales,sales_detrend,is_anomaly
1598,2015-06-15,TX_1,FOODS,10036.83,6405.333363,True
6251,2012-04-01,TX_2,FOODS,9969.98,6396.227490,True
6257,2012-04-07,TX_2,FOODS,10122.12,6548.441950,True
6286,2012-05-06,TX_2,FOODS,10007.18,6433.861843,True
6783,2013-09-15,TX_2,FOODS,10379.24,6812.089655,True


In [157]:
df.loc[df['is_anomaly'], 'sales_detrend'] = np.nan

<br />

### 3.4 Check for Missing Sales Values

In [158]:
df.isna().sum()

date                0
store_id            0
cat_id              0
sales               0
sales_detrend    1623
is_anomaly          0
dtype: int64

The only attribute that has missing value is the `sales_detrend` which is the outlier been detected earlier.

<br />

### 3.5 Impute Missing Value

In [159]:
df['sales_detrend'] = df['sales_detrend'].interpolate()

<br />

---

## 4.0 Save Processed Data

In [161]:
df.drop(['sales_detrend', 'is_anomaly'], axis=1).to_csv('sales_processed.csv', index=False)